# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нам нужно обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Требование к модели: значение метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Библиотеки-и-настройки" data-toc-modified-id="Библиотеки-и-настройки-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Библиотеки и настройки</a></span></li><li><span><a href="#Нормализация-текстов" data-toc-modified-id="Нормализация-текстов-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Нормализация текстов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Разделение-выборок" data-toc-modified-id="Разделение-выборок-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Разделение выборок</a></span></li><li><span><a href="#Модели" data-toc-modified-id="Модели-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Модели</a></span><ul class="toc-item"><li><span><a href="#Модель-1:-spaCy→TF-IDF→LR" data-toc-modified-id="Модель-1:-spaCy→TF-IDF→LR-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Модель 1: spaCy→TF-IDF→LR</a></span></li><li><span><a href="#Модель-2:-spaCy→TF-IDF→LGBM" data-toc-modified-id="Модель-2:-spaCy→TF-IDF→LGBM-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Модель 2: spaCy→TF-IDF→LGBM</a></span></li><li><span><a href="#Модель-3:-BERT-и-CatBoost" data-toc-modified-id="Модель-3:-BERT-и-CatBoost-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Модель 3: BERT и CatBoost</a></span></li><li><span><a href="#Результаты" data-toc-modified-id="Результаты-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Результаты</a></span></li></ul></li></ul></div>

## Библиотеки и настройки

In [1]:
import math

import numpy as np
import pandas as pd
import time as t
import spacy
import en_core_web_sm
import re

import matplotlib.pyplot as plt
import seaborn as sns

!pip install CatBoost -q
!pip install lightgbm -q
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from nltk.corpus import stopwords
import spacy

!pip install transformers -q
import transformers
import torch

from tqdm.auto import tqdm

plt.style.use('seaborn')
pd.set_option('display.max_rows', None, 'display.max_columns', None)

     |████████████████████████████████| 76.6 MB 67.4 MB/s 
     |████████████████████████████████| 5.5 MB 15.2 MB/s 
     |████████████████████████████████| 7.6 MB 58.5 MB/s 
     |████████████████████████████████| 182 kB 70.7 MB/s 


In [2]:
tqdm.pandas()
print("Setup Complete")

Setup Complete


##Загрузка данных

In [3]:
df_reviews = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [4]:
df_reviews.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Датасет содержит 3 столбца: 
- `toxic` целевой признак текста, указывающий на токсичность
- `text` текст сообщения пользователя
- `Unnamed: 0` дубль к индексу, видимо артефакт от выгрузки данных

Сразу избавимся столбца, не нужного в рамках проекта

In [5]:
df_reviews.drop('Unnamed: 0',axis=1,inplace=True)

In [6]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [7]:
df_reviews.toxic.value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

 - Пропусков в данных нет
- Классы не сбалансированы, это может повлиять на обучение модели

## Нормализация текстов

Для моделей нужны тексты в нижнем регистре без цифр и знаков пунктуации. 

Обработаем тексты в соответствии с этими требованиями

In [8]:
corpus = df_reviews['text']

In [9]:
def clear_text(text):

    clean_text = re.sub(r'[^a-zA-z\']', ' ', text.lower())
    clean_text = " ".join(clean_text.split())

    return clean_text

In [10]:
df_reviews['text_norm'] = corpus.apply(lambda x: clear_text(x))

## Лемматизация

In [11]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [12]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
def text_preprocessing(text):
    
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc]
    
    return ' '.join(tokens)

In [14]:
corpus_preprocessed = corpus.progress_apply(text_preprocessing)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [15]:
nlp = en_core_web_sm.load(disable=['parser', 'ner'])

def lemmatize(text):

    doc = nlp(text)

    lemmas = []
    for token in doc:
        lemmas.append(token.lemma_)

    return ' '.join(lemmas)

In [16]:
corpus_lemm = corpus_preprocessed.progress_apply(lemmatize)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [17]:
df_reviews['review_lemm'] = corpus_lemm

## Разделение выборок

In [18]:
features = df_reviews.review_lemm
target = df_reviews.toxic

In [19]:
X_train, X_test, y_train, y_test = train_test_split(features,target,
                                                    test_size=.2,
                                                    random_state=42)

In [20]:
y_train.shape, y_test.shape

((127433,), (31859,))

In [21]:
tf_idf_lemm = TfidfVectorizer(stop_words=stop_words)
tfidf_train_lemm = tf_idf_lemm.fit_transform(X_train)
tfidf_test_lemm = tf_idf_lemm.transform(X_test)

## Модели

### Модель 1: spaCy→TF-IDF→LR


In [22]:
lr = LogisticRegression(random_state=42, solver='liblinear')
lr_fit_time = t.time()
lr.fit(tfidf_train_lemm, y_train)
lr_fit_time = t.time() - lr_fit_time
lr_pred = lr.predict(tfidf_test_lemm)
lr_fit_time/60

0.027907768885294598

In [23]:
lr_f1score = metrics.f1_score(y_test,lr_pred)
lr_rocauc = metrics.roc_auc_score(y_test,lr_pred)
lr_acc = metrics.accuracy_score(y_test,lr_pred)
lr_f1score, lr_rocauc, lr_acc

(0.7355887355887356, 0.8014551226870992, 0.9560877616999905)

### Модель 2: spaCy→TF-IDF→LGBM

In [24]:
lgbm = LGBMClassifier(random_state=42)
lgbm_fit_time = t.time()
lgbm.fit(tfidf_train_lemm, y_train)
lgbm_fit_time = t.time() - lgbm_fit_time
lgbm_pred = lgbm.predict(tfidf_test_lemm)
lgbm_fit_time/60

1.1917658805847169

In [25]:
lgbm_f1score = metrics.f1_score(y_test,lgbm_pred)
lgbm_rocauc = metrics.roc_auc_score(y_test,lgbm_pred)
lgbm_acc = metrics.accuracy_score(y_test,lgbm_pred)
lgbm_f1score, lgbm_rocauc, lgbm_acc

(0.7475745927146258, 0.8151244543777746, 0.9567155277943439)

###  Модель 3: BERT и CatBoost

In [26]:
tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert')
config = transformers.BertConfig.from_pretrained('unitary/toxic-bert')
model = transformers.BertModel.from_pretrained('unitary/toxic-bert')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
def BERT_text_to_embeddings(texts, max_length=512, batch_size=100, force_device=None, disable_progress_bar=False):
    
    ids_list = []
    attention_mask_list = []

    # text to padded ids of tokens along with their attention masks
        
    tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert')
    ids_list = texts.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=max_length))
    
    ids_list = ids_list.apply(lambda x: np.array(x[:max_length] + [0]*(max_length - len(x))))
    attention_mask_list = ids_list.apply(lambda x: np.where(x != 0, 1, 0))
    
    if force_device is not None:
        device = torch.device(force_device)
    else:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    model.to(device)
    if not disable_progress_bar:
        print(f'Using the {device} device.')
    
    # Эмбендинги получаем батчами

    embeddings = []

    for i in tqdm(range(math.ceil(len(ids_list)/batch_size)), disable=disable_progress_bar):
            
        ids_batch = torch.LongTensor(ids_list[batch_size*i:batch_size*(i+1)].reset_index(drop=True)).to(device)
        attention_mask_batch = torch.LongTensor(attention_mask_list[batch_size*i:batch_size*(i+1)].reset_index(drop=True)).to(device)
    
        with torch.no_grad():            
            model.eval()
            batch_embeddings = model(input_ids=ids_batch, attention_mask=attention_mask_batch)   
        embeddings.append(batch_embeddings[0][:,0,:].detach().cpu().numpy())
        
    return np.concatenate(embeddings)

In [28]:
bert_train_features = BERT_text_to_embeddings(X_train, batch_size=10)

Using the cuda device.


  0%|          | 0/12744 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


In [29]:
bert_test_features = BERT_text_to_embeddings(X_test, batch_size=10)

Using the cuda device.


  0%|          | 0/3186 [00:00<?, ?it/s]

In [30]:
bert_cb = CatBoostClassifier(random_state=42,
                             verbose=False)

bert_cb_fit_time = t.time()

bert_cb.fit(bert_train_features, y_train)

bert_cb_fit_time = t.time() - bert_cb_fit_time
bert_cb_fit_time/60

6.523801835378011

In [31]:
bert_cb_pred_time = t.time()

bert_cb_pred = bert_cb.predict(bert_test_features)

bert_cb_pred_time = t.time() - bert_cb_pred_time
bert_cb_pred_time

0.9573788642883301

In [32]:
cb_f1score = metrics.f1_score(y_test,bert_cb_pred)
cb_rocauc = metrics.roc_auc_score(y_test,bert_cb_pred)
cb_acc = metrics.accuracy_score(y_test,bert_cb_pred)
cb_f1score,cb_rocauc,cb_acc

(0.931390977443609, 0.9607222309466983, 0.986251922533664)

### Результаты

In [33]:
results = pd.DataFrame({
    'Model': ['spaCy→TF-IDF→LR', 'spaCy→TF-IDF→LGBM', 'BERT и CatBoost'],
    'f1 score': [lr_f1score, lgbm_f1score, cb_f1score],
    'ROC AUC':[lr_rocauc,lgbm_rocauc,cb_rocauc],
    'Accuracy':[lr_acc,lgbm_acc,cb_acc],
    'Fit_time':[lr_fit_time,lgbm_fit_time,bert_cb_fit_time],
    })

results.sort_values(by='f1 score', ascending=False)

,Model,f1 score,ROC AUC,Accuracy,Fit_time
2,BERT и CatBoost,0.931391,0.960722,0.986252,391.428110
1,spaCy→TF-IDF→LGBM,0.747575,0.815124,0.956716,71.505953
0,spaCy→TF-IDF→LR,0.735589,0.801455,0.956088,1.674466


По результатам работы связка BERT и CatBoost посказала лучшее значение метрики f1 равный 0,93. Она рекомендована к использованию

Остальные модели не прошли порог в 0,75. Возможно можно повыситить точность подбором порога. В данный момент нет возможности пересчитать проект (закончилось время на GPU в Colab 🤷‍♂️)

# Заключение


Задачей проекта было получить модель для определения токсичности комментариев и их разделения на **токсичные** и **не токсичные**.

Как изначальный критерий для выбора модели использовался показатель метрики **f1 не ниже 0,75**

Было построено 3 модели с 2 подходами к лемматизации текста:
- spaCy+TFidf+LogisticRegression
- spaCy+TFidf+LGBMClassifier
- Предобученная BERT и CatBoostClassifier

По результатам работы к использования рекомендована модель с использованием BERT и CatBoost. Она одна преодолела порог f1>0,75 и получила значение 0,90